<h1> Przygotowanie i podział danych

In [1]:
# Importy
from ucimlrepo import fetch_ucirepo
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score

In [2]:
# Pobranie danych
heart_disease = fetch_ucirepo(id=45)

# Porzucenie linii z pustymi etykietami oraz odpowiadajacych im wartosci
feature_matrix = heart_disease.data.features.dropna()
labels = heart_disease.data.targets.loc[feature_matrix.index]

# Przetworzenie zbioru wartości przewidywanych do wartości binarnych
y_binary = labels.copy()
y_binary['num'] = y_binary['num'].apply(lambda x: 1 if x != 0 else 0)                      

# Utworznnie zbioru dummy etykiet
x_dummy = pd.get_dummies(feature_matrix, columns=['cp', 'restecg', 'slope','ca','thal'])         

# Podział danych na zbiór uczący i testowy
x_train, x_test, y_train, y_test = train_test_split(x_dummy, y_binary, test_size=0.2, random_state=268555)

# Normalizacja cech trenignowych i testowych
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

# Konwersja danych do wymaganego formatu
x_train = np.array(x_train).astype(float)
y_train = np.array(y_train).astype(float)


<h1> Definicja funkcji wykorzystywanych w zadaniu

In [3]:
def sigmoid(z):
    """
    Poniższa funkcja różni się od tej zaproponowanej w liście,
    ze względu na błędy obliczeniowe przy ujemnych wartościach parametru z.
    Funkcja where jest implementacją operatora ternarnego biblioteki numpy
    """
    return np.where(z >= 0, 
                1 / (1 + np.exp(-z)), 
                np.exp(z) / (1 + np.exp(z)))

def compute_gradient(feature_matrix, labels, weights):
    """
    feature_matrix:  macierz wejściowa cech o wymiarze (ilość rekordów, ilość cech)
    labels:         lista prawdziwych etykiet o długości równej ilości rekordów
    weights:        lista wag poszczególnych cech. Długość jest równa ilości cech
    """
    numeber_of_records = feature_matrix.shape[0]
    gradients = np.zeros(weights.shape) # Inicjalizacja gradientów
    
    # Obliczanie prognoz modelu
    z = np.matmul(feature_matrix, weights)
    p = sigmoid(z)
    
    # Obliczanie gradientu dla każdej cechy
    for i in range(len(weights)):
        gradients[i] = -np.sum((labels - p) * feature_matrix[:, i]) / numeber_of_records
    
    return gradients

# Funkcja gradientu spadku dla aktualizacji wag
def gradient_descent(featureMatrix, labels, weights, learning_rate = 0.1, epochs = 1000, min_loss_change = 0.001):
    """
    feature_matrix:     macierz wejściowa cech o wymiarze (ilość rekordów, ilość cech)
    labels:             lista prawdziwych etykiet o długości równej ilości rekordów
    weights:            lista wag poszczególnych cech. Długość jest równa ilości cech
    learning_rate:      współczynnik uczenia
    epochs:             liczba epok
    min_loss_change:    minimalna różnica potrzebna do kontynuowania obliczeń
    """

    prev_loss = 0

    for epoch in range(epochs):
        gradients = compute_gradient(featureMatrix, labels, weights)    # Obliczanie gradientów dla każdej epoki
        weights -= learning_rate * gradients                            # Aktualizacja wag
        
        # Wyświetlanie informacji o postępie
        if epoch % 100 == 0:
            epsilon = 1e-5  # mała wartość dla uniknięcia log(0)
            p = np.clip(sigmoid(featureMatrix @ weights), epsilon, 1 - epsilon) 
            """
            funkcja clip z zastosowaniem zmiennej epsilon zapobiega przekazaniu wartości 0 oraz 1,
            do funkcji sigmoid, co zapobiega obliczaniu logarytmu z tych wartości.
            """
            loss = np.mean(-labels * np.log(p) - (1 - labels) * np.log(1 - p))

            print(f'Epoch {epoch}: Loss = {"%.4f" % loss}')

            if (loss <= prev_loss + min_loss_change):
                break

            prev_loss = loss
    
    return weights


<h1> Trenowanie modelu oraz weryfikacja otrzymanych wyników

In [4]:
weights = np.random.rand(x_train.shape[1]) # Inicjalizacja wag jako lista wartości z akresu 0-1

# Trening modelu
trained_w = gradient_descent(x_train, y_train, weights)

Epoch 0: Loss = 4.2863
Epoch 100: Loss = 4.9855
Epoch 200: Loss = 5.0059
Epoch 300: Loss = 5.0197
Epoch 400: Loss = 5.0291
Epoch 500: Loss = 5.0356
Epoch 600: Loss = 5.0399
Epoch 700: Loss = 5.0428
Epoch 800: Loss = 5.0449
Epoch 900: Loss = 5.0462


In [5]:
# Prognozy prawdopodobieństwa
y_pred = sigmoid(np.dot(x_test, trained_w))
# Zamiana prawdopodobieństw na klasy (y < 0.5 => y = 0, y >= 0.5 => y = 1)
y_pred = (y_pred >= 0.5).astype(int)

# Wyliczenie metryk
print("Accuracy:", "%.2f" % accuracy_score(y_test, y_pred))
print("F1 Score:", "%.2f" % f1_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))

Accuracy: 0.82
F1 Score: 0.79
Precision: 0.75
Recall: 0.84
